In [2]:
""" 线性回归的简洁实现 """

' 线性回归的简洁实现 '

In [3]:
# 通过使用深度学习框架来简洁地实现 线性回归模型 生成数据集
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

In [4]:
# 调用框架中现有的 API 来读取数据
def load_array(data_arrays, batch_size, is_train=True):
    """ 
    构造一个 PyTorch 数据迭代器 
    :param data_arrays: 包含特征和标签的元组（如 (features, labels)）
    :param batch_size: 每个批次的样本数量
    :param is_train: 是否为训练模式（决定是否打乱数据顺序）
    :return: PyTorch 数据加载器对象
    """
    
    # 将特征和标签组合成 PyTorch 数据集对象
    # *data_arrays 解包元组，例如将 (features, labels) 解为 features, labels
    dataset = data.TensorDataset(*data_arrays)
    
    # 创建数据加载器，按批次加载数据
    # shuffle=is_train：训练时打乱数据顺序，验证/测试时保持顺序
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

# 获取数据迭代器的第一个批次数据
# iter(data_iter) 创建迭代器，next() 获取下一个批次
next(iter(data_iter))

[tensor([[-0.2650,  0.5514],
         [-0.5000, -1.0541],
         [ 1.7470, -0.5217],
         [ 2.7687, -0.4720],
         [-0.7131,  0.4301],
         [ 0.9517,  0.3629],
         [-1.2120,  1.8238],
         [ 1.8306, -1.4740],
         [ 0.0329,  0.8826],
         [-0.2995,  1.6272]]),
 tensor([[ 1.7933],
         [ 6.8012],
         [ 9.4629],
         [11.3368],
         [ 1.3220],
         [ 4.8610],
         [-4.4370],
         [12.8669],
         [ 1.2570],
         [-1.9507]])]

In [5]:
# 使用框架的预定义好的层
# `nn`是神经网络的缩写
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))

In [7]:
# 初始化模型参数
net[0].weight.data.normal_(0, 0.01)  # net[0] 表示访问 nn.Sequential 容器中的 第一个层
net[0].bias.data.fill_(0)

tensor([0.])

In [8]:
# 计算均方误差使用的是MSELoss类
loss = nn.MSELoss()

In [9]:
# 实例化 SGD 实例
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

In [13]:
# 训练过程
num_epochs = 3  # 定义训练的总轮数（完整遍历数据集的次数）
for epoch in range(num_epochs):
    # 内层循环：逐批次遍历训练数据
    for X, y in data_iter:
        # 前向传播：计算当前批次的预测值与真实值的损失
        l = loss(net(X), y)
        
        # 梯度清零：清除优化器中上一次迭代的梯度，避免梯度累积
        trainer.zero_grad()
        
        # 反向传播：根据损失计算模型参数的梯度
        l.backward()
        
        # 参数更新：根据梯度使用优化器（SGD）更新模型参数
        trainer.step()
    
    # 每轮训练结束后，计算整个数据集上的损失
    # 注意：此处使用全部数据（features, labels）而非小批量，用于评估模型整体性能
    l = loss(net(features), labels)
    
    # 打印当前轮数和总损失值
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000092
epoch 2, loss 0.000091
epoch 3, loss 0.000092
